# Function Programming SIG 12.8.22

In [ ]:
:opt no-lint
import Data.List
import Data.Char

## Functors

In [ ]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

In [ ]:
data Pair a = Pair a a deriving Show

In [ ]:
instance Functor Pair where
    fmap myFunc (Pair x y) = Pair (myFunc x) (myFunc y)

In [ ]:
fmap reverse (Pair "foo" "bar")

In [ ]:
fmap (*100) (Just 5)
fmap (*100) Nothing

## Monads

In [ ]:
class Monad m where
    (>>=) :: m a -> (a -> m b) -> m b
    return :: a -> m a

In [ ]:
instance Monad Maybe where
-- (>>=) :: Maybe a -> (a -> Maybe b) -> Maybe b
    Nothing >>= f = Nothing
    Just x >>= f x
    return x = Just x

In [ ]:
access :: Int -> [a] -> Maybe a
access i xs = if i >= 0 && i < length xs then Just (xs !! i) else Nothing

access 0 [1..10]
access 10 [1..10]

In [ ]:
access 0 [[[1,2], [3,4], [5,6]], []]

In [ ]:
access 2 (access 0 [[[1,2], [3,4], [5,6]], []])

In [ ]:
fmap (access 2) (access 0 [[[1,2], [3,4], [5,6]], []])

In [ ]:
fmap (access 0) (Just (Just [5,6]))

In [ ]:
access 0 [[[1,2], [3,4], [5,6]], []] >>= access 2

In [ ]:
access 0 [[[1,2], [3,4], [5,6]], []] >>= access 2 >>= access 0

In [ ]:
access 0 [[[1,2], [3,4], [5,6]], []] >>= access 2 >>= access 0

In [ ]:
return [[[1,2], [3,4], [5,6]], []] >>= access 0 >>= access 2 >>= access 0

### The IO Monad

In [ ]:
:t putStrLn
:t getLine
-- (>>=) :: IO a -> (a -> IO b) -> IO b

In [ ]:
foo :: IO ()
foo = putStrLn "foo"

foo

In [ ]:
bar :: IO ()
bar = putStrLn "please enter your name" 
    >>= (\_ -> getLine 
    >>= (\x -> putStrLn $ "your name is " ++ x))

bar

In [ ]:
bar' :: IO ()
bar' = do
    putStrLn "please enter your name"
    x <- getLine
    putStrLn $ "your name is " ++ x
    
bar'

### The List Monad

In [ ]:
instance Monad [] where
    x >>= f = concat (map f x)
    return x = [x]
-- (>>=) :: [a] -> (a -> [b]) -> [b]

In [ ]:
garble :: [a] -> [a]
garble xs = do
    x <- reverse xs
    y <- [x,x] 
    return y
    
garble "hello world"
garble [1,2,3,4]

In [ ]:
garble' :: [a] -> [a]
garble' xs = reverse xs >>= (\x -> [x,x])

garble' "hello world"

In [ ]:
garble'' :: [a] -> [a]
garble'' = concat . map (\x -> [x,x]) . reverse

garble'' "hello world"

### Monads as Composition

In [ ]:
(>=>) :: Monad m => (a -> m b) -> (b -> m c) -> (a -> m c)
f >=> g = \x -> f x >>= g

In [ ]:
(.) :: (b -> c) -> (a -> b) -> (a -> c)
g . f = \x -> g (f x)

In [ ]:
divide :: Double -> Double -> Maybe Double
divide _ 0 = Nothing
divide x y = Just $ x / y

In [ ]:
(reverse . map toUpper) "hello world"

In [ ]:
(access 0 >=> divide 3.14) [90.9, 89.32, 0]

### Monad Laws

Left identity: return >=> f = f  
Right identity: g >=> return = g  
Associativity: (f >=> g) >=> h = f >=> (g >=> h)  

### Parsing with Megaparsec

[Tutorial](https://markkarpov.com/tutorial/megaparsec.html)